In [1]:
import pandas as pd

In [2]:
filepath = './datasets/'

df_movies = pd.read_csv(filepath+'df_movies_clean.csv')

df_cast = pd.read_csv(filepath+'df_cast.csv')

df_crew = pd.read_csv(filepath+'df_crew.csv')

In [3]:

# CANTIDAD_FILMACiONES_MES
def cantidad_filmaciones_mes( mes:str ):
  '''
  Se ingresa un mes en idioma Español.
  Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.

  Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X
  '''
  month_map = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12
  }

  month_number = month_map.get(mes.lower())

  # Set the release_date column to a datetime format
  df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d')
  # Filter the movies that were released in the specified month and count them
  month_movies = df_movies[df_movies['release_date'].dt.month == month_number]
  month_count = len(month_movies)

  answer = {'mes':mes.lower(), 'cantidad':month_count}
  return answer

# CANTIDAD_FILMACiONES_DIA
def cantidad_filmaciones_dia( dia:str ):
  '''
  Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X
  '''
  day_map = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miércoles': 'Wednesday',
    'miercoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sábado': 'Saturday',
    'sabado': 'Saturday',
    'domingo': 'Sunday',
  }

  day = day_map.get(dia.lower())
  # Set the release_date column to a datetime format
  df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d')

  # Count the number of movies that were released on Mondays
  day_count = len(df_movies[df_movies['release_date'].dt.day_name() == day])

  answer = {'dia':dia.lower(), 'cantidad':day_count}
  return answer

# SCORE_TITULO
def score_titulo( titulo:str ):
  '''
  Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
  Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X
  '''

  # Filter the DataFrame to include only the row(s) that match the title
  movie = df_movies[df_movies['title'].str.lower() == titulo.lower()]

  # Extract the values for the title, release_year, and popularity columns
  movie_title = movie['title'].values[0]
  release_year = movie['release_year'].values[0]
  popularity = movie['popularity'].values[0]

  # Return the movie title, release year, and popularity
  answer = {'titulo':movie_title, 'anio':release_year, 'popularidad':popularity}
  return answer


# VOTOS_TITULO
def votos_titulo(titulo:str): 
  '''
  Se ingresa el título de una filmación esperando como respuesta:
  * el título,
  * la cantidad de votos
  * el valor promedio de las votaciones

  La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.

  Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X
  '''

  # Filter the DataFrame to include only the row(s) that match the title
  movie = df_movies[df_movies['title'].str.lower() == titulo.lower()]

  # Extract the values for the title, release_year, and popularity columns
  movie_title = movie['title'].values[0]
  release_year = movie['release_year'].values[0]
  vote_average = movie['vote_average'].values[0]
  vote_count = movie['vote_count'].values[0]

  if vote_count >= 2000.0:
    # Return the movie title, release year, and votes
    answer = {'titulo':movie_title,
              'anio':release_year,
              'voto_total':vote_count,
              'voto_promedio':vote_average}
    
  else:
      answer = {'mensaje': f'La película "{movie_title}" no cumple con la condición de tener al menos 2000 valoraciones.'}
    
    
  return answer



# GET_ACTOR
def get_actor( nombre_actor:str ):

  # Filter the cast DataFrame to include only rows for the given actor
  actor_df = df_cast[df_cast['name'].str.lower() == nombre_actor.lower()]

  # Join the actor_df with the movies_df on the 'id' column

  joined_df = actor_df.merge(df_movies,
                            left_on='movie_id',
                            right_on='id')

  # Calculate the sum and average of the 'return' column
  return_sum = joined_df['return'].sum()
  movies_count = joined_df['return'].count()
  average_return = return_sum / movies_count

  actor_name = actor_df['name'].iloc[0]

  answer =  {'actor':actor_name,
             'cantidad_filmaciones':movies_count,
             'retorno_total':return_sum,
             'retorno_promedio':average_return}
  return answer


# GET_DIRECTOR
def get_director( nombre_director:str ):
  # Filter the crew DataFrame to include only rows for the given director
  director_df = df_crew[df_crew['name'].str.lower() == nombre_director.lower()]

  director_df = director_df[director_df['job'] == 'Director']

  # Join the director_df with the movies_df on the 'id' column
  joined_df = director_df.merge(df_movies,
                          left_on='movie_id',
                          right_on='id')

  # Calculate the sum of the 'return' column
  return_sum = joined_df['return'].sum()

  # Calculate the number of movies directed by the director
  movie_count = director_df['movie_id'].count()

  # Calculate the average return per movie
  average_return = return_sum / movie_count

  # Return a DataFrame with the movie name, release date, return, budget, and revenue
  director_name = director_df['name'].iloc[0]
  movie_data_df = joined_df[['title', 'release_year', 'return', 'budget', 'revenue']]
  
  # Convert each row to a dictionary
  dict_list = movie_data_df.to_dict(orient='index')

  # Rename the keys in each dictionary
  for key in dict_list.keys():
      dict_list[key] = {
          'titulo': dict_list[key]['title'],
          'año': dict_list[key]['release_year'],
          'retorno': dict_list[key]['return'],
          'budget': dict_list[key]['budget'],
          'revenue': dict_list[key]['revenue']
      }
      dict_list[key].pop('title')
      dict_list[key].pop('release_year')
      dict_list[key].pop('return')
      dict_list[key].pop('budget')
      dict_list[key].pop('revenue')


  answer = {'director':director_name,
            'retorno_total_director':return_sum,
            'peliculas': dict_list}
  return answer

In [5]:
cantidad_filmaciones_mes('marzo')

{'mes': 'marzo', 'cantidad': 3553}

In [6]:
cantidad_filmaciones_mes('ABRiL')

{'mes': 'abril', 'cantidad': 3453}

In [7]:
cantidad_filmaciones_dia('miercoles')

{'dia': 'miercoles', 'cantidad': 7035}

In [8]:
score_titulo('toy story')

{'titulo': 'Toy Story', 'anio': 1995, 'popularidad': 21.946943}

In [9]:
score_titulo('juMANji')

{'titulo': 'Jumanji', 'anio': 1995, 'popularidad': 17.015539}

In [10]:
print(votos_titulo('toy story'))

{'titulo': 'Toy Story', 'anio': 1995, 'voto_total': 5415.0, 'voto_promedio': 7.7}


In [11]:
print(votos_titulo('Grumpier Old Men'))

{'mensaje': 'La película "Grumpier Old Men" no cumple con la condición de tener al menos 2000 valoraciones.'}


In [12]:
print(get_actor('Wallace Shawn'))

{'actor': 'Wallace Shawn', 'cantidad_filmaciones': 75, 'retorno_total': 50.0185446127092, 'retorno_promedio': 0.6669139281694559}


In [4]:
get_director('John Lasseter')

KeyError: 'title'

In [14]:
import pickle

pickle.dump(df_movies.to_dict,open('df_movies_dict.pkl','wb'))
pickle.dump(df_cast.to_dict,open('df_cast_dict.pkl','wb'))
pickle.dump(df_crew.to_dict,open('df_crew_dict.pkl','wb'))

In [6]:
nombre_director = 'John Lasseter'
# Filter the crew DataFrame to include only rows for the given director
director_df = df_crew[df_crew['name'].str.lower() == nombre_director.lower()]

director_df = director_df[director_df['job'] == 'Director']

# Join the director_df with the movies_df on the 'id' column
joined_df = director_df.merge(df_movies,
                        left_on='movie_id',
                        right_on='id')

# Calculate the sum of the 'return' column
return_sum = joined_df['return'].sum()

# Calculate the number of movies directed by the director
movie_count = director_df['movie_id'].count()

# Calculate the average return per movie
average_return = return_sum / movie_count

# Return a DataFrame with the movie name, release date, return, budget, and revenue
director_name = director_df['name'].iloc[0]
movie_data_df = joined_df[['title', 'release_year', 'return', 'budget', 'revenue']]
# Rename the columns
movie_data_df.columns = ['titulo', 'anio', 'retorno_pelicula', 'budget_pelicula', 'revenue_pelicula']
# Convert each row to a dictionary
dict_list = movie_data_df.to_dict(orient='index')

answer = {'director':director_name,
        'retorno_total_director':return_sum,
        'peliculas': dict_list}

answer

{'director': 'John Lasseter',
 'retorno_total_director': 27.65437835777778,
 'peliculas': {0: {'titulo': 'Toy Story',
   'anio': 1995,
   'retorno_pelicula': 12.4518011,
   'budget_pelicula': 30000000,
   'revenue_pelicula': 373554033.0},
  1: {'titulo': "A Bug's Life",
   'anio': 1998,
   'retorno_pelicula': 3.027157158333333,
   'budget_pelicula': 120000000,
   'revenue_pelicula': 363258859.0},
  2: {'titulo': 'Toy Story 2',
   'anio': 1999,
   'retorno_pelicula': 5.526298544444445,
   'budget_pelicula': 90000000,
   'revenue_pelicula': 497366869.0},
  3: {'titulo': 'Luxo Jr.',
   'anio': 1986,
   'retorno_pelicula': 0.0,
   'budget_pelicula': 0,
   'revenue_pelicula': 0.0},
  4: {'titulo': 'Cars',
   'anio': 2006,
   'retorno_pelicula': 3.849859575,
   'budget_pelicula': 120000000,
   'revenue_pelicula': 461983149.0},
  5: {'titulo': 'Cars 2',
   'anio': 2011,
   'retorno_pelicula': 2.79926198,
   'budget_pelicula': 200000000,
   'revenue_pelicula': 559852396.0},
  6: {'titulo': 'Ti